In [1]:
from pyspark.sql import SparkSession
import numpy as np
import pyspark.sql.functions as func

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1558441724271_0001,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .config("spark.default.parallelism","20") \
    .config("spark.executor.memory","16g") \
    .config("spark.driver.memory","16g") \
    .appName("Stage One") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

VBox()

In [3]:
# Load the orginal dataset
rawdata = "s3://amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
raw = spark.read.csv(rawdata,header=True,sep='\t')

VBox()

In [4]:
raw.printSchema()

VBox()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)

In [5]:
#the total number of reviews
raw.count()

VBox()

4751577

In [6]:
#the number of unique users
raw.select('customer_id').distinct().count()

VBox()

1940732

In [8]:
#the number of unique products
raw.select('product_id').distinct().count()

VBox()

782326

In [9]:
#the largest number of reviews published by a single user
raw.groupby('customer_id').count().orderBy('count', ascending=False).limit(1).show()

VBox()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   50736950| 7168|
+-----------+-----+

In [10]:
#the top 10 users ranked by the number of reviews they publish
top_10_users = raw.groupby('customer_id').count().orderBy('count', ascending=False).limit(10).show()

VBox()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   50736950| 7168|
|   38214553| 5412|
|   51184997| 5369|
|   18116317| 4222|
|   23267387| 4023|
|   50345651| 3793|
|   14539589| 2896|
|   15725862| 2842|
|   19380211| 2592|
|   20018062| 2568|
+-----------+-----+

In [13]:
#the median number of reviews published by a user
raw.groupby('customer_id').count().approxQuantile('count',[0.5],0.05)[0]

VBox()

1.0

In [14]:
#the largest number of reviews written for a single product
raw.groupby('product_id').count().orderBy('count', ascending=False).limit(1).show()

VBox()

+----------+-----+
|product_id|count|
+----------+-----+
|B00008OWZG| 3936|
+----------+-----+

In [15]:
#the top 10 products ranked by the number of reviews they have
raw.groupby('product_id').count().orderBy('count', ascending=False).limit(10).show()

VBox()

+----------+-----+
|product_id|count|
+----------+-----+
|B00008OWZG| 3936|
|B0000AGWEC| 3326|
|B00MIA0KGY| 2699|
|B00NEJ7MMI| 2420|
|B000089RVX| 2376|
|B004EBT5CU| 2106|
|B0026P3G12| 2080|
|B00009PRZF| 2026|
|B00004XONN| 1901|
|B00006J6VG| 1793|
+----------+-----+

In [16]:
#the median number of reviews a product has
raw.groupby('product_id').count().approxQuantile('count',[0.5],0.05)[0]

VBox()

2.0